In [1]:
from signal import signal
from matplotlib.offsetbox import HPacker
import scipy
import scipy
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from spectral_connectivity import Multitaper
from spectral_connectivity import Connectivity
from spectral_connectivity import multitaper_connectivity
from pathlib import Path
import pandas as pd
from scipy.signal import decimate

function to calculate delays at theta gamma frequencies

In [2]:
# this function gives the list of theta and gamma delays calculated using windowing 

def theta_beta(pathsHPC,pathsPFC):
    time_halfbandwidth_product = 5
    n_trials = len(pathsHPC)
    theta=[]

    for i in range(n_trials):
        dataHPC = scipy.io.loadmat(pathsHPC[i])
        dataPFC = scipy.io.loadmat(pathsPFC[i])

        current_dataHPC = dataHPC['HPC']
        current_dataPFC = dataPFC['PFC']
        n_time_samples = len(current_dataHPC)

        
        downsampled_dataHPC = decimate(current_dataHPC, 4, ftype='fir')
        downsampled_dataPFC = decimate(current_dataPFC, 4, ftype='fir')

        
        n_time_samples_downsampled = len(downsampled_dataHPC)

        time = np.arange(0, n_time_samples_downsampled) / (2500 / 4)
        signal1 = downsampled_dataHPC
        signal2 = downsampled_dataPFC

        signals = np.stack((signal1, signal2), axis=-1)
    
        m = Multitaper(
        signals,
        sampling_frequency=625,
        time_halfbandwidth_product=time_halfbandwidth_product,
        start_time=time[0],
        time_window_duration=10,
        time_window_step=1
        )

        c = Connectivity(fourier_coefficients=m.fft(), frequencies=m.frequencies, time=m.time)

        delay_theta, slope, r_value= c.group_delay(
        frequencies_of_interest=list(range(5,12)), frequency_resolution=m.frequency_resolution
        )
        
        D1=np.nanmean(delay_theta[:,0,1]) #hippocampus as leading signal

        theta.append(D1)
    return theta

In [3]:
#this function calculate the mean and standard error of mean of theta and gamma delays calculated in theta_gamma function

def average(theta,beta):
    # Calculate the average
    theta_mean = np.nanmean(theta)
    beta_mean = np.nanmean(beta)


    # Calculate the standard error of mean
    t=pd.Series(theta)
    g=pd.Series(beta)
    theta_std = (np.nanstd(theta))/(np.sqrt(t.count()))
    beta_std = (np.nanstd(beta))/(np.sqrt(g.count()))

    return theta_mean,theta_std

In [4]:

pathsHPC = [path for path in Path('/Users/omid/Desktop/2021-06-03_13-38-14_posttrial5').resolve().glob("**/*HPC*.mat")] # works (all full paths to HPC files)
pathsPFC = [path for path in Path('/Users/omid/Desktop/2021-06-03_13-38-14_posttrial5').resolve().glob("**/*PFC*.mat")] # works (all full paths to PFC files)


In [5]:
pathsPFC

[PosixPath('/Users/omid/Desktop/2021-06-03_13-38-14_posttrial5/PFC_100_CH22_0.continuous.mat')]

In [6]:
theta = theta_beta(pathsHPC,pathsPFC)
theta_mean,theta_std = average(theta)

: 